In [4]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

In [5]:
# Postgres info to connect

connection_args = {
  "host": "twitchdata.chd4n5ul8muk.us-east-2.rds.amazonaws.com",
  "user": "postgres",
    "password":"FwwBFmleh65qYxKxDVb9",
  "port": 5432,
  "dbname": "twitchdata"
}


connection = pg.connect(**connection_args)

In [6]:
query = "SELECT COUNT(DISTINCT(stream_id)) FROM stream_data;"

pd_sql.read_sql(query, connection)

,count
0,214100


In [6]:
query = "SELECT COUNT(DISTINCT(game_id)) FROM game_information;"

pd_sql.read_sql(query, connection)

,count
0,465


In [11]:
query = "SELECT COUNT(DISTINCT(game_name)) FROM game_genres;"

pd_sql.read_sql(query, connection)

,count
0,1001


### Getting the data for surprise recommender in the format needed to be funneled into the "surprise_test" notebook. ###

In [7]:
query = '''SELECT user_name, game_name, max(viewer_count)
FROM general_view
GROUP BY user_name, game_name
ORDER BY user_name DESC, max DESC'''

surprise_grid = pd_sql.read_sql(query, connection)

In [9]:
#surprise_grid.to_csv('user_game_grid.csv', index = False)

### Querying the complete dataset for EDA as the new data comes in throughout the day. ###

In [13]:
query = '''SELECT stream_id, user_id, user_name, stream_data.game_id, stream_type, title, viewer_count, started_at, language, time_logged, game_information.game_name, pic_url, game_genres.game_genres 
FROM stream_data 
LEFT JOIN game_information ON stream_data.game_id = game_information.game_id
LEFT JOIN game_genres ON game_information.game_name = game_genres.game_name;'''

complete_data = pd_sql.read_sql(query, connection)

In [14]:
data.info()

,stream_id,user_id,user_name,game_id,stream_type,title,viewer_count,started_at,language,time_logged,game_name,pic_url,game_genres
0,34366704544,190582624,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24,Tibia,https://static-cdn.jtvnw.net/ttv-boxart/Tibia-...,RPG
1,34366704544,190582624,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24,Tibia,https://static-cdn.jtvnw.net/ttv-boxart/Tibia-...,Adventure Game
2,34366704544,190582624,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24,Tibia,https://static-cdn.jtvnw.net/ttv-boxart/Tibia-...,MMORPG


In [15]:
#complete_data.to_csv('<date>_merged_data.csv')

In [3]:
#complete_data = pd.read_csv('<date>_merged_data.csv')